## Utilities

In [1]:
#!fsharp
module Utilities =
    let rec combinations accumulator desiredListSize possibleListItems = seq {
        match desiredListSize, possibleListItems with
        | currentListSize, firstListElement::remainderOfList ->
            if currentListSize > 0 then yield! combinations (firstListElement::accumulator) (currentListSize - 1) remainderOfList
            if currentListSize >= 0 then yield! combinations accumulator currentListSize remainderOfList
        | 0, [] -> yield accumulator
        | _, [] -> ()
    }

## Day 1
Before you leave, the Elves in accounting just need you to fix your expense report (your puzzle input); apparently, something isn't quite adding up.

Specifically, they need you to find the two entries that sum to 2020 and then multiply those two numbers together.

For example, suppose your expense report contained the following:

1721
979
366
299
675
1456
In this list, the two entries that sum to 2020 are 1721 and 299. Multiplying them together produces 1721 * 299 = 514579, so the correct answer is 514579.

Of course, your expense report is much larger. Find the two entries that sum to 2020; what do you get if you multiply them together?

### Shared Setup Code

In [1]:
#!fsharp
let input = 
    [
        1939; 1585; 1712; 1600; 1370; 1447; 1247; 1446; 1323; 1713; 1277; 1946; 1677; 1428; 1231; 1481; 1976; 1709; 1508; 1668; 1302; 77; 
        1351; 1605; 1999; 1982; 1583; 1756; 1957; 1624; 1745; 1938; 1784; 1403; 1642; 1691; 569; 1762; 1555; 1937; 1383; 1897; 1334; 1965;
        1683; 1475; 1776; 1791; 1707; 1987; 1233; 1416; 1769; 1345; 1874; 1255; 1744; 1944; 1404; 1360; 1304; 1417; 1977; 1656; 790; 1788;
        1353; 1296; 1673; 1810; 1684; 1742; 1425; 1887; 1444; 1352; 1229; 1414; 1493; 1402; 1947; 1669; 1412; 1531; 1474; 1637; 1314; 1607;
        1829; 1923; 1949; 1757; 1307; 1714; 1748; 1550; 1372; 1615; 1235; 1272; 1408; 1749; 1687; 1613; 1528; 1561; 341; 1308; 1660; 1667;
        1313; 1991; 1675; 1394; 1704; 1303; 1440; 1592; 1857; 1752; 1839; 1397; 1699; 1426; 1878; 1759; 1814; 1096; 372; 1596; 1500; 1774;
        1627; 1696; 1851; 1020; 1819; 1292; 1616; 1672; 1279; 1543; 1526; 1682; 1568; 1582; 1921; 922; 1773; 1482; 1238; 1973; 1517; 1909;
        409; 1634; 1468; 1445; 1801; 1631; 1407; 1820; 1603; 1495; 1333; 1241; 1849; 82; 1339; 1413; 90; 1662; 1291; 1740; 1340; 1365; 2003;
        1546; 1621; 1650; 1518; 1807; 1382; 1433; 1968; 1940; 1986; 1437; 1651; 1237; 1862; 1409; 1200; 2002; 2009; 1735; 1487; 1706; 1643; 1505; 
    ]

### Part 1

In [1]:
#!fsharp
seq {
    for x in input do 
        let y = 2020 - x
        if input |> List.contains y then yield sprintf $"Answer: x: {x} y: {y}, x * y : {x * y}"
}
|> Seq.head 

Answer: x: 1938 y: 82, x * y : 158916

### Part 2

In [1]:
#!fsharp
seq {
    for x in input do
        for y in input do 
            let z = 2020 - (x + y)
            if input |> List.contains z then yield $"Answer: x: {x} y: {y}, z: {z}, x * y * z : {x * y * z}"
}
|> Seq.head

Answer: x: 1307 y: 341, z: 372, x * y * z : 165795564

## Day 2

Your flight departs in a few days from the coastal airport; the easiest way down to the coast from here is via toboggan.

The shopkeeper at the North Pole Toboggan Rental Shop is having a bad day. "Something's wrong with our computers; we can't log in!" You ask if you can take a look.

Their password database seems to be a little corrupted: some of the passwords wouldn't have been allowed by the Official Toboggan Corporate Policy that was in effect when they were chosen.

To try to debug the problem, they have created a list (your puzzle input) of passwords (according to the corrupted database) and the corporate policy when that password was set.

For example, suppose you have the following list:

1-3 a: abcde
1-3 b: cdefg
2-9 c: ccccccccc
Each line gives the password policy and then the password. The password policy indicates the lowest and highest number of times a given letter must appear for the password to be valid. For example, 1-3 a means that the password must contain a at least 1 time and at most 3 times.

In the above example, 2 passwords are valid. The middle password, cdefg, is not; it contains no instances of b, but needs at least 1. The first and third passwords are valid: they contain one a or nine c, both within the limits of their respective policies.

How many passwords are valid according to their policies?

### Input

In [1]:
#!fsharp
open System.Text.RegularExpressions
open System.IO

type PasswordRow =
    {
        LowerLimit: int
        UpperLimit: int
        TargetCharacter: char
        AttemptedPassword: char []
    }

let day2Input = File.ReadAllText("./Day2Input.txt")
let pattern = @"(?<lower>\d+)-(?<upper>\d+) (?<target>\w){1}: (?<password>\w+)"
let rows =
    Regex.Matches(day2Input, pattern, RegexOptions.Multiline)
    |> Seq.cast<Match>
    |> Seq.map (fun m -> 
        {
            LowerLimit = m.Groups.["lower"].Value |> int
            UpperLimit = m.Groups.["upper"].Value |> int
            TargetCharacter = m.Groups.["target"].Value |> char
            AttemptedPassword = m.Groups.["password"].Value.ToCharArray()
        })

### Part 1

In [1]:
#!fsharp
let validPassword passwordRow =
    let targetCharCount = 
        passwordRow.AttemptedPassword 
        |> Array.filter (fun c -> c = passwordRow.TargetCharacter)
        |> Array.length
    targetCharCount >= passwordRow.LowerLimit && targetCharCount <= passwordRow.UpperLimit

let validRows = 
    rows
    |> Seq.filter validPassword
    |> Seq.length

validRows

418

### Part 2

While it appears you validated the passwords correctly, they don't seem to be what the Official Toboggan Corporate Authentication System is expecting.

The shopkeeper suddenly realizes that he just accidentally explained the password policy rules from his old job at the sled rental place down the street! The Official Toboggan Corporate Policy actually works a little differently.

Each policy actually describes two positions in the password, where 1 means the first character, 2 means the second character, and so on. (Be careful; Toboggan Corporate Policies have no concept of "index zero"!) Exactly one of these positions must contain the given letter. Other occurrences of the letter are irrelevant for the purposes of policy enforcement.

Given the same example list from above:

1-3 a: abcde is valid: position 1 contains a and position 3 does not.
1-3 b: cdefg is invalid: neither position 1 nor position 3 contains b.
2-9 c: ccccccccc is invalid: both position 2 and position 9 contain c.
How many passwords are valid according to the new interpretation of the policies?

In [1]:
#!fsharp
let validTobogganCompanyPassword passwordRow = 
    let position1Valid = passwordRow.AttemptedPassword.[passwordRow.LowerLimit - 1] = passwordRow.TargetCharacter
    let position2Valid = passwordRow.AttemptedPassword.[passwordRow.UpperLimit - 1] = passwordRow.TargetCharacter
    position1Valid && not position2Valid || position2Valid && not position1Valid

let validRows = 
    rows 
    |> Seq.filter validTobogganCompanyPassword
    |> Seq.length

validRows

616

## Day 3

With the toboggan login problems resolved, you set off toward the airport. While travel by toboggan might be easy, it's certainly not safe: there's very minimal steering and the area is covered in trees. You'll need to see which angles will take you near the fewest trees.

Due to the local geology, trees in this area only grow on exact integer coordinates in a grid. You make a map (your puzzle input) of the open squares (.) and trees (#) you can see. For example:

```
..##.......
#...#...#..
.#....#..#.
..#.#...#.#
.#...##..#.
..#.##.....
.#.#.#....#
.#........#
#.##...#...
#...##....#
.#..#...#.#
```

These aren't the only trees, though; due to something you read about once involving arboreal genetics and biome stability, the same pattern repeats to the right many times:

```
..##.........##.........##.........##.........##.........##.......  --->
#...#...#..#...#...#..#...#...#..#...#...#..#...#...#..#...#...#..
.#....#..#..#....#..#..#....#..#..#....#..#..#....#..#..#....#..#.
..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#
.#...##..#..#...##..#..#...##..#..#...##..#..#...##..#..#...##..#.
..#.##.......#.##.......#.##.......#.##.......#.##.......#.##.....  --->
.#.#.#....#.#.#.#....#.#.#.#....#.#.#.#....#.#.#.#....#.#.#.#....#
.#........#.#........#.#........#.#........#.#........#.#........#
#.##...#...#.##...#...#.##...#...#.##...#...#.##...#...#.##...#...
#...##....##...##....##...##....##...##....##...##....##...##....#
.#..#...#.#.#..#...#.#.#..#...#.#.#..#...#.#.#..#...#.#.#..#...#.#  --->
```

You start on the open square (.) in the top-left corner and need to reach the bottom (below the bottom-most row on your map).

The toboggan can only follow a few specific slopes (you opted for a cheaper model that prefers rational numbers); start by counting all the trees you would encounter for the slope right 3, down 1:

From your starting position at the top-left, check the position that is right 3 and down 1. Then, check the position that is right 3 and down 1 from there, and so on until you go past the bottom of the map.

The locations you'd check in the above example are marked here with O where there was an open square and X where there was a tree:

```
..##.........##.........##.........##.........##.........##.......  --->
#..O#...#..#...#...#..#...#...#..#...#...#..#...#...#..#...#...#..
.#....X..#..#....#..#..#....#..#..#....#..#..#....#..#..#....#..#.
..#.#...#O#..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#
.#...##..#..X...##..#..#...##..#..#...##..#..#...##..#..#...##..#.
..#.##.......#.X#.......#.##.......#.##.......#.##.......#.##.....  --->
.#.#.#....#.#.#.#.O..#.#.#.#....#.#.#.#....#.#.#.#....#.#.#.#....#
.#........#.#........X.#........#.#........#.#........#.#........#
#.##...#...#.##...#...#.X#...#...#.##...#...#.##...#...#.##...#...
#...##....##...##....##...#X....##...##....##...##....##...##....#
.#..#...#.#.#..#...#.#.#..#...X.#.#..#...#.#.#..#...#.#.#..#...#.#  --->
```

### Input

In [1]:
#!fsharp
open System.IO

let hill = File.ReadAllLines("./Day3Input.txt")

### Part 1


In this example, traversing the map using this slope would cause you to encounter 7 trees.

Starting at the top-left corner of your map and following a slope of right 3 and down 1, how many trees would you encounter?

In [1]:
#!fsharp
let mutable treeCount = 0
let mutable currentRow = 0
let mutable currentColumn = 0

while currentRow < hill.Length do
    let columnToCheck =
        if currentColumn >= hill.[currentRow].Length then currentColumn - hill.[currentRow].Length
        else currentColumn
    if hill.[currentRow].[columnToCheck] = '#' then
        treeCount <- treeCount + 1
    
    currentRow <- currentRow + 1
    currentColumn <- columnToCheck + 3

treeCount

220

### Part 2

Time to check the rest of the slopes - you need to minimize the probability of a sudden arboreal stop, after all.

Determine the number of trees you would encounter if, for each of the following slopes, you start at the top-left corner and traverse the map all the way to the bottom:

Right 1, down 1.
Right 3, down 1. (This is the slope you already checked.)
Right 5, down 1.
Right 7, down 1.
Right 1, down 2.
In the above example, these slopes would find 2, 7, 3, 4, and 2 tree(s) respectively; multiplied together, these produce the answer 336.

What do you get if you multiply together the number of trees encountered on each of the listed slopes?

In [1]:
#!fsharp
type TraversalPattern =
    {
        Horizontal : int
        Vertical : int
    }

let checkSlope pattern =
    let mutable treeCount = 0
    let mutable currentRow = 0
    let mutable currentColumn = 0

    while currentRow < hill.Length do
        let columnToCheck =
            if currentColumn >= hill.[currentRow].Length then currentColumn - hill.[currentRow].Length
            else currentColumn
        if hill.[currentRow].[columnToCheck] = '#' then
            treeCount <- treeCount + 1
        
        currentRow <- currentRow + pattern.Vertical
        currentColumn <- columnToCheck + pattern.Horizontal

    treeCount

let patterns =
    [
        { Horizontal = 1; Vertical = 1 }
        { Horizontal = 3; Vertical = 1 }
        { Horizontal = 5; Vertical = 1 }
        { Horizontal = 7; Vertical = 1 }
        { Horizontal = 1; Vertical = 2 }
    ]

patterns 
|> List.map checkSlope 
|> List.reduce (*)

2138320800

## Day 4

You arrive at the airport only to realize that you grabbed your North Pole Credentials instead of your passport. While these documents are extremely similar, North Pole Credentials aren't issued by a country and therefore aren't actually valid documentation for travel in most of the world.

It seems like you're not the only one having problems, though; a very long line has formed for the automatic passport scanners, and the delay could upset your travel itinerary.

Due to some questionable network security, you realize you might be able to solve both of these problems at the same time.

The automatic passport scanners are slow because they're having trouble detecting which passports have all required fields. The expected fields are as follows:

byr (Birth Year)
iyr (Issue Year)
eyr (Expiration Year)
hgt (Height)
hcl (Hair Color)
ecl (Eye Color)
pid (Passport ID)
cid (Country ID)
Passport data is validated in batch files (your puzzle input). Each passport is represented as a sequence of key:value pairs separated by spaces or newlines. Passports are separated by blank lines.

Here is an example batch file containing four passports:

```
ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
byr:1937 iyr:2017 cid:147 hgt:183cm

iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
hcl:#cfa07d byr:1929

hcl:#ae17e1 iyr:2013
eyr:2024
ecl:brn pid:760753108 byr:1931
hgt:179cm

hcl:#cfa07d eyr:2025 pid:166559648
iyr:2011 ecl:brn hgt:59in
```

The first passport is valid - all eight fields are present. The second passport is invalid - it is missing hgt (the Height field).

The third passport is interesting; the only missing field is cid, so it looks like data from North Pole Credentials, not a passport at all! Surely, nobody would mind if you made the system temporarily ignore missing cid fields. Treat this "passport" as valid.

The fourth passport is missing two fields, cid and byr. Missing cid is fine, but missing any other field is not, so this passport is invalid.

According to the above rules, your improved system would report 2 valid passports.

### Input

In [1]:
#!fsharp
open System.IO
open System

(*
    Start off by splitting the input into the distinct groups (assuming that the groups are split by double newlines),
    then cleaning up, splitting, and sorting the information for each group
*) 
let day4input = 
    File.ReadAllText("./Day4input.txt").Split($"{Environment.NewLine}{Environment.NewLine}")
    |> Array.map (fun x -> x.Replace(Environment.NewLine, " ").Replace("  ", " ") .Split(" ") |> Array.map (fun y -> y.Trim()) |> Array.sort )

### Part 1

Count the number of valid passports - those that have all required fields. Treat cid as optional. In your batch file, how many passports are valid?

In [1]:
#!fsharp
let requiredAttributes = [ "byr"; "iyr"; "eyr"; "hgt"; "hcl"; "ecl"; "pid"; ]

let isValidPart1 (input: string []) =
    // Discard the values and keep the headers for now, as part 1 only requires us to verify that a heading is there
    let combinedAttributes =
        input 
        |> Array.map (fun x -> x.Split(":") |> Array.head)
    
    // Verify that the newly-created list contains all of the required attributes
    requiredAttributes
    |> List.forall (fun x -> combinedAttributes |> Array.contains x)
    
day4input
|> Array.map (fun x -> x |> isValidPart1)
// We're dealing with bools here, so we can just use the identity function rather than do any other kind of filtering
|> Array.filter id
|> Array.length

213

### Part 2

The line is moving more quickly now, but you overhear airport security talking about how passports with invalid data are getting through. Better add some data validation, quick!

You can continue to ignore the cid field, but each other field has strict rules about what values are valid for automatic validation:

byr (Birth Year) - four digits; at least 1920 and at most 2002.
iyr (Issue Year) - four digits; at least 2010 and at most 2020.
eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
hgt (Height) - a number followed by either cm or in:
If cm, the number must be at least 150 and at most 193.
If in, the number must be at least 59 and at most 76.
hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
pid (Passport ID) - a nine-digit number, including leading zeroes.
cid (Country ID) - ignored, missing or not.
Your job is to count the passports where all required fields are both present and valid according to the above rules. Here are some example values:

```
byr valid:   2002
byr invalid: 2003

hgt valid:   60in
hgt valid:   190cm
hgt invalid: 190in
hgt invalid: 190

hcl valid:   #123abc
hcl invalid: #123abz
hcl invalid: 123abc

ecl valid:   brn
ecl invalid: wat

pid valid:   000000001
pid invalid: 0123456789
```
Here are some invalid passports:

```
eyr:1972 cid:100
hcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926

iyr:2019
hcl:#602927 eyr:1967 hgt:170cm
ecl:grn pid:012533040 byr:1946

hcl:dab227 iyr:2012
ecl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277

hgt:59cm ecl:zzz
eyr:2038 hcl:74454a iyr:2023
pid:3556412378 byr:2007
Here are some valid passports:

pid:087499704 hgt:74in ecl:grn iyr:2012 eyr:2030 byr:1980
hcl:#623a2f

eyr:2029 ecl:blu cid:129 byr:1989
iyr:2014 pid:896056539 hcl:#a97842 hgt:165cm

hcl:#888785
hgt:164cm byr:2001 iyr:2015 cid:88
pid:545766238 ecl:hzl
eyr:2022

iyr:2010 hgt:158cm hcl:#b6652a ecl:blu byr:1944 eyr:2021 pid:093154719
```

Count the number of valid passports - those that have all required fields and valid values. Continue to treat cid as optional. In your batch file, how many passports are valid?

In [1]:
#!fsharp
type IdentityData =
    {
        original: string
        byr: int
        ecl: string
        eyr: int
        hcl: string
        hgt: int
        iyr: int
        pid: string
    }

let between low high input =
    input >= low && input <= high

let tryCreateIdentityData original byr ecl eyr hcl hgt hgtUnit iyr pid =
    let validByr = between 1920 2002 byr
    let validEyr = between 2020 2030 eyr
    let validHgt =
        if hgtUnit = "cm" then between 150 193 hgt
        else if hgtUnit = "in" then between 59 76 hgt
        else false
    let validIyr = between 2010 2020 iyr

    match (validByr, validEyr, validHgt, validIyr) with
    | (true, true, true, true) -> 
        Some
            {
                original = original
                byr = byr
                ecl = ecl
                eyr = eyr
                hcl = hcl
                hgt = hgt
                iyr = iyr
                pid = pid
            }
    | _ -> None

let parseRow inputRow =
    let matches = Regex.Match(inputRow, @"^byr:(?<byr>\d{4}) (cid:(?<cid>\d*) )?ecl:(?<ecl>amb|blu|brn|gry|grn|hzl|oth) eyr:(?<eyr>\d{4}) hcl:(?<hcl>#[a-f0-9]{6}) hgt:(?<hgt>\d{2,3})(?<hgtUnit>cm|in) iyr:(?<iyr>\d{4}) pid:(?<pid>\d{9})$")
    if matches.Success then
        let byr = matches.Groups.["byr"].Value |> int
        let ecl = matches.Groups.["ecl"].Value
        let eyr = matches.Groups.["eyr"].Value |> int
        let hcl = matches.Groups.["hcl"].Value
        let hgt = matches.Groups.["hgt"].Value |> int
        let hgtUnit = matches.Groups.["hgtUnit"].Value
        let iyr = matches.Groups.["iyr"].Value |> int
        let pid = matches.Groups.["pid"].Value
        
        tryCreateIdentityData inputRow byr ecl eyr hcl hgt hgtUnit iyr pid
        // Some
        //     {
        //         original = inputRow
        //         byr = byr
        //         ecl = ecl
        //         eyr = eyr
        //         hcl = hcl
        //         hgt = hgt
        //         iyr = iyr
        //         pid = pid
        //     }
    else None
    
day4input
|> Array.map (fun x -> x |> Array.reduce (fun y z -> $"{y} {z}"))
// |> Array.reduce (fun x y -> $"{x}{Environment.NewLine}{y}")
|> Array.choose parseRow
|> Array.length


147

## Day 5

You board your plane only to discover a new problem: you dropped your boarding pass! You aren't sure which seat is yours, and all of the flight attendants are busy with the flood of people that suddenly made it through passport control.

You write a quick program to use your phone's camera to scan all of the nearby boarding passes (your puzzle input); perhaps you can find your seat through process of elimination.

Instead of zones or groups, this airline uses binary space partitioning to seat people. A seat might be specified like FBFBBFFRLR, where F means "front", B means "back", L means "left", and R means "right".

The first 7 characters will either be F or B; these specify exactly one of the 128 rows on the plane (numbered 0 through 127). Each letter tells you which half of a region the given seat is in. Start with the whole list of rows; the first letter indicates whether the seat is in the front (0 through 63) or the back (64 through 127). The next letter indicates which half of that region the seat is in, and so on until you're left with exactly one row.

For example, consider just the first seven characters of FBFBBFFRLR:

Start by considering the whole range, rows 0 through 127.
F means to take the lower half, keeping rows 0 through 63.
B means to take the upper half, keeping rows 32 through 63.
F means to take the lower half, keeping rows 32 through 47.
B means to take the upper half, keeping rows 40 through 47.
B keeps rows 44 through 47.
F keeps rows 44 through 45.
The final F keeps the lower of the two, row 44.
The last three characters will be either L or R; these specify exactly one of the 8 columns of seats on the plane (numbered 0 through 7). The same process as above proceeds again, this time with only three steps. L means to keep the lower half, while R means to keep the upper half.

For example, consider just the last 3 characters of FBFBBFFRLR:

Start by considering the whole range, columns 0 through 7.
R means to take the upper half, keeping columns 4 through 7.
L means to take the lower half, keeping columns 4 through 5.
The final R keeps the upper of the two, column 5.
So, decoding FBFBBFFRLR reveals that it is the seat at row 44, column 5.

Every seat also has a unique seat ID: multiply the row by 8, then add the column. In this example, the seat has ID 44 * 8 + 5 = 357.

Here are some other boarding passes:

BFFFBBFRRR: row 70, column 7, seat ID 567.
FFFBBBFRRR: row 14, column 7, seat ID 119.
BBFFBBFRLL: row 102, column 4, seat ID 820.

### Input & Shared Model

In [1]:
#!fsharp
let day5input = 
    File.ReadAllLines("./Day5Input.txt")
    |> Array.map (fun x -> x.ToCharArray())

let (|Front|Back|Left|Right|) character =
    if character = 'F' then Front
    else if character = 'B' then Back
    else if character = 'L' then Left
    else Right

type Range = 
    {
        Lower : int
        Upper : int
    }
    static member DefaultRowRange =
        {
            Lower = 0
            Upper = 127
        }
    static member DefaultColumnRange =
        {
            Lower = 0
            Upper = 7
        }

let rec reduceRange range (input: char list) =
    match input with
    | [] -> 0
    | [lastItem] -> 
        match lastItem with 
        | Front | Left -> range.Lower
        | Back | Right -> range.Upper
    | firstItem::remainderOfList -> 
        let halfwayPoint = (range.Lower + range.Upper) / 2
        let range =
            match firstItem with
            | Front | Left  ->
                {
                    Lower = range.Lower
                    Upper = halfwayPoint
                }
            | Back | Right -> 
                {
                    // Round up when taking the upper half
                    Lower = halfwayPoint + 1
                    Upper = range.Upper
                }
        reduceRange range remainderOfList



type InputRow =
    {
        RowInput : char list
        ColumnInput : char list
    }

type Seat =
    {
        Row : int
        Column : int
    }
    member this.SeatId() = 
        (this.Row * 8) + this.Column

### Part 1

As a sanity check, look through your list of boarding passes. What is the highest seat ID on a boarding pass?

In [1]:
#!fsharp
day5input
|> Array.map (fun x -> 
    {
        RowInput = x.[0..6] |> Array.toList
        ColumnInput = x.[7..9] |> Array.toList
    })
|> Array.map (fun x -> 
    let row = x.RowInput |> reduceRange Range.DefaultRowRange
    let column = x.ColumnInput |> reduceRange Range.DefaultColumnRange

    {
        Row = row
        Column = column
    })
|> Array.map (fun x -> x.SeatId())
|> Array.max

955

### Part 2

Ding! The "fasten seat belt" signs have turned on. Time to find your seat.

It's a completely full flight, so your seat should be the only missing boarding pass in your list. However, there's a catch: some of the seats at the very front and back of the plane don't exist on this aircraft, so they'll be missing from your list as well.

Your seat wasn't at the very front or back, though; the seats with IDs +1 and -1 from yours will be in your list.

What is the ID of your seat?